In [ ]:
import torch
from torchvision import datasets, transforms
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import numpy as np
from sklearn.metrics import f1_score

In [ ]:
# Проверяем количество доступных GPU
gpu_count = torch.cuda.device_count()
print(f"Количество доступных GPU: {gpu_count}")

# Если GPU доступен, выводим информацию о первом устройстве
if gpu_count > 0:
    print(f"Информация о GPU 0: {torch.cuda.get_device_name(0)}")
    print(f"Объём памяти GPU 0: {torch.cuda.get_device_properties(0).total_memory / 1e9} GB")
else:
    print("GPU не доступен.")

In [ ]:
# Устанавливаем преобразования для тренировочных и тестовых данных
transform = transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor()])

# Загрузка тренировочного и тестового датасетов
train_dataset = datasets.Flowers102(root='data', split='train', download=True, transform= transform)
validation_dataset = datasets.Flowers102(root='data', split='val', download=True, transform= transform)
test_dataset = datasets.Flowers102(root='data', split='test', download=True, transform= transform)

In [ ]:
print(f"Train dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(validation_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")

In [ ]:
train_set = test_dataset
combined_train_set = torch.utils.data.ConcatDataset([train_dataset, validation_dataset])

In [ ]:
print(len(train_set))
print(len(combined_train_set))

In [ ]:
txt_file = open('flowers102_label_names.txt','r').read().splitlines()
class_names = [x.replace('"','').replace("'", '') for x in txt_file]

In [ ]:
import matplotlib.pyplot as plt

def imshow(img, mean, std):
    # Декодируем изображение обратно с учетом нормализации
    for c in range(img.shape[0]):  # Для каждого канала
        img[c] = img[c] * std[c] + mean[c]  # Обратная нормализация для каждого канала

    img = np.clip(img, 0, 1)  # Ограничиваем значения пикселей в пределах [0, 1]
    plt.imshow(img.transpose((1, 2, 0)))  # Переход от (C, H, W) к (H, W, C)
    plt.axis('off')  # Отключаем оси
    plt.show()

# Извлекаем изображение из набора данных
img_tensor = train_set[0][0]  # Получаем первое изображение (и его метку)

# Нормализационные параметры для обратной нормализации
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])

# Визуализируем изображение
imshow(img_tensor.numpy(), mean, std)

In [ ]:
fig, axis = plt.subplots(nrows=2, ncols=5, figsize=(15, 5))

# В цикле проходим по всем осям и отображаем случайные изображения
for ax in axis.ravel():
    # Генерируем случайный индекс
    idx = np.random.randint(len(train_set))
    
    # Получаем изображение и метку
    img_tensor, label = train_set[idx]
    
    # Обратная нормализация
    img_tensor = img_tensor * np.array([0.229, 0.224, 0.225])[:, None, None] + np.array([0.485, 0.456, 0.406])[:, None, None]
    
    # Преобразуем тензор из (C, H, W) в (H, W, C)
    img = img_tensor.permute(1, 2, 0).numpy()
    
    # Отображаем изображение
    ax.imshow(np.clip(img, 0, 1))  # Ограничиваем значения пикселей в диапазоне [0, 1]
    
    # Название класса
    ax.set_title(class_names[label], fontsize=14)
    
    # Отключаем оси
    ax.axis('off')

# Показать изображение
plt.tight_layout()
plt.show()

In [ ]:
train_dataloader = DataLoader(train_set, batch_size=30, shuffle=True, drop_last=True)
val_loader = DataLoader(validation_dataset, batch_size=30, shuffle=False)
test_dataloader = DataLoader(combined_train_set, batch_size=30, shuffle=False,drop_last=False)

In [ ]:
# Пример получения одного батча из train_loader
data_iter = iter(train_dataloader)
images, labels = next(data_iter)
print(images.shape, labels.shape)

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        # Слои свёрточной нейронной сети
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1)  # Слой 1
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)  # Слой 2

        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)  # Слой 3
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)  # Слой 4
        
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)  # Слой 5
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)  # Слой 6
        
        self.flatten = nn.Flatten()  # Слой 7
        
        self.fc1 = nn.Linear(in_features=128 * 28 * 28, out_features=512)  # Слой 8 (128 * 16 * 16 - размер после свёрток и пулингов)
        
        
        self.fc2 = nn.Linear(in_features=512, out_features=102)  # Слой 9

        self.dropout = nn.Dropout(p=0.5)
        
    def forward(self, x):
        # Прямой проход через сеть
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool1(x)

        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool2(x)

        x = self.conv3(x)
        x = F.relu(x)
        x = self.pool3(x)

        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)

        x = self.fc2(x)
        return x

# Пример создания модели
model = CNN()
print(model)

In [ ]:
!pip install torchinfo

In [ ]:
from torchinfo import summary
summary(CNN(), input_size=(30,3, 224, 224))

In [ ]:
!pip install tensorboard

In [ ]:
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter 

In [ ]:
# Создаём модель
model = CNN()

# Убираем использование GPU (если доступен)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Оптимизатор
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)  # SGD с моментумом

# Функция потерь для многоклассовой классификации
criterion = nn.CrossEntropyLoss()

# Логирование с использованием TensorBoard
writer = SummaryWriter(log_dir='runs/flower_classification')  # Создаём директорию для TensorBoard


In [ ]:
train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []

num_epochs = 10  # Количество эпо

for epoch in range(num_epochs):
    model.train()
    
    running_loss = 0.0
    correct_preds = 0
    total_preds = 0
    
    for inputs, labels in train_dataloader:  # train_loader — это твой DataLoader для тренировочных данных
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()  # Обнуляем градиенты

        outputs = model(inputs)  # Прямой проход
        loss = criterion(outputs, labels)  # Вычисляем потерю

        loss.backward()  # Обратное распространение
        optimizer.step()  # Шаг оптимизации

        running_loss += loss.item() * inputs.size(0)  # Накопление потерь
        _, predicted = torch.max(outputs, 1)  # Выбираем класс с максимальной вероятностью
        correct_preds += (predicted == labels).sum().item()
        total_preds += labels.size(0)

    # Логирование потерь и точности в TensorBoard
    epoch_loss = running_loss / len(train_dataloader.dataset)
    epoch_accuracy = correct_preds / total_preds
    train_losses.append(epoch_loss)
    train_accuracies.append(epoch_accuracy)

    writer.add_scalar('Training Loss', epoch_loss, epoch)
    writer.add_scalar('Training Accuracy', epoch_accuracy, epoch)

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.4f}')

 # Шаг валидации
    model.eval()  # Переключаем модель в режим оценки (выключаем Dropout и BatchNorm, если есть)
    
    val_loss = 0.0
    correct_preds_val = 0
    total_preds_val = 0

    with torch.no_grad():  # Отключаем вычисление градиентов для валидации
        for inputs, labels in val_loader:  # val_loader — это твой DataLoader для валидационных данных
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)  # Прямой проход
            loss = criterion(outputs, labels)  # Вычисляем потерю

            val_loss += loss.item() * inputs.size(0)  # Накопление потерь
            _, predicted = torch.max(outputs, 1)  # Выбираем класс с максимальной вероятностью
            correct_preds_val += (predicted == labels).sum().item()
            total_preds_val += labels.size(0)

    # Логирование потерь и точности на валидации в TensorBoard
    val_loss = val_loss / len(val_loader.dataset)
    val_accuracy = correct_preds_val / total_preds_val
    val_losses.append(val_loss)
    val_accuracies.append(val_accuracy)
    
    writer.add_scalar('Validation Loss', val_loss, epoch)
    writer.add_scalar('Validation Accuracy', val_accuracy, epoch)

    print(f'Epoch {epoch+1}/{num_epochs}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}')

# Закрытие TensorBoard writer после тренировки
writer.close()

In [ ]:
# Визуализация потерь
plt.figure(figsize=(12, 6))

# Потери
plt.subplot(1, 2, 1)
plt.plot(range(1, num_epochs + 1), train_losses, label='Training Loss', color='blue', marker='o')
plt.plot(range(1, num_epochs + 1), val_losses, label='Validation Loss', color='red', marker='x')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Точность
plt.subplot(1, 2, 2)
plt.plot(range(1, num_epochs + 1), train_accuracies, label='Training Accuracy', color='blue', marker='o')
plt.plot(range(1, num_epochs + 1), val_accuracies, label='Validation Accuracy', color='red', marker='x')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Отображаем графики
plt.tight_layout()
plt.show()